# Importações

In [ ]:
import os
import pandas as pd
from google.cloud import storage

# Ambiente de Execução: Colab X VSCode

In [ ]:
# # Usar no Google Colab
# from google.colab import auth
# auth.authenticate_user()
# path_folder_bronze = '/content/'
# path_folder_silver = '/content/silver/'


In [ ]:
# Usar no VSCode
path_folder_bronze = os.path.abspath('../data/temp/bronze')
path_folder_silver = os.path.abspath('../data/temp/silver')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../gcp_key.json"

# Leitura, filtro e limpeza dos dados
- Objetivo: Manter os dados de chegada em cada ponto e calcular o atraso ou adiatamento em relação ao horário previsto

## Informações Iniciais

In [ ]:
def extract_year_month(date): # Função para extrair o ano e o mês de uma data
    return date.strftime('%y%m')

year_month = extract_year_month(pd.to_datetime('2017-06-01'))  # Exemplo de uso da função, como se fosse o dia 01 de junho de 2017
print(f'Year-Month: {year_month}')

In [ ]:
def read_csv(path_folder_bronze, file_name_bronze):
    # Devido a limitação de memória, o arquivo CSV é lido em pedaços (chunks)
    return pd.read_csv(os.path.join(path_folder_bronze, file_name_bronze), on_bad_lines='skip', chunksize=10000)

file_name_bronze = f'mta_{year_month}.csv'

In [ ]:
def show_first_chunk(path_folder_bronze, file_name_bronze): # Exibir o cabeçalho do primeiro pedaço (chunk) do DataFrame
    for chunk in read_csv(path_folder_bronze, file_name_bronze):
        display(chunk.head())
        display(chunk.info())
        break

# Função para exibir o cabeçalho do primeiro pedaço do arquivo CSV
show_first_chunk(path_folder_bronze, file_name_bronze)

## Filtro e seleção de colunas
- Selecionar das colunas úteis
- Remover de linhas null em Horário Programado de Chegada no Ponto
- Filtrar apenas registros que estão no Ponto 

In [ ]:
def clean_and_filter_data(path_folder_bronze, file_name_bronze):
    df_list_chunks = []  # Lista vazia para guardar os pedaços do dataset filtrado e limpo

    for chunk in read_csv(path_folder_bronze, file_name_bronze):
        df_temp = chunk[['RecordedAtTime', 'DirectionRef', 'PublishedLineName', 'VehicleRef',
                        'ArrivalProximityText', 'ScheduledArrivalTime']]  # Colunas que serão utilizadas
        df_temp = df_temp[(df_temp['ArrivalProximityText'] == 'at stop')] # Filtro de registros no ponto
        df_temp.dropna(subset=['ScheduledArrivalTime'], inplace=True)  # Remoção de valores null
        df_temp.drop(['ArrivalProximityText'], axis=1, inplace=True) # Remove a coluna do ponto pois não será usada
        df_list_chunks.append(df_temp)
        
    df = pd.concat(df_list_chunks, ignore_index=True) # Concatena todos os pedaços em um único DataFrame   
    print(f'Dados do arquivo "{file_name_bronze}" carregados e filtrados.') 
    return df
    
df = clean_and_filter_data(path_folder_bronze, file_name_bronze)
display(df.head(15))
display(df.info())

## Adição de colunas
- Data do Registro
- Horário Previsto e Realizado no Ponto
- Diferença entre Previsto e Realizado
- Faixa Horária Prevista e Realizada

In [ ]:
def add_RecordedAtDate_and_Time_columns(df):
    df['RecordedAtDate'] = pd.to_datetime(df['RecordedAtTime']).dt.date # Cria uma nova coluna com a data
    df['RecordedAtTime'] = pd.to_datetime(df['RecordedAtTime']).dt.strftime('%H:%M:%S') # Transforma a coluna de timestamp em hora

    df['RecordedAtDate'] = pd.to_datetime(df['RecordedAtDate'], format='%Y-%m-%d') # Converte a coluna de data para datetime
    df['RecordedAtTime'] = pd.to_datetime(df['RecordedAtTime'], format='%H:%M:%S') # Converte a coluna de data e hora para datetime
    df['RecordedAtTime'] = pd.to_timedelta(df['RecordedAtTime'].dt.strftime('%H:%M:%S')) # Extrai apenas o tempo da coluna de data e hora

    print('Coluna de data e hora realizadas adicionadas.')
    return df

df = add_RecordedAtDate_and_Time_columns(df)
display(df.head(15))
display(df.info())

In [ ]:
def fix_ScheduledArrivalTime(hora_str):
    hora = int(hora_str[:2])
    if hora > 23:
        return "00:" + hora_str[3:]
    return hora_str

def fix_ScheduledArrivalTime_column(df):
    df['ScheduledArrivalTime'] = df['ScheduledArrivalTime'].apply(fix_ScheduledArrivalTime)

    df['ScheduledArrivalTime'] = pd.to_datetime(df['ScheduledArrivalTime'], format='%H:%M:%S') # Converte a coluna de data e hora para datetime
    df['ScheduledArrivalTime'] = pd.to_timedelta(df['ScheduledArrivalTime'].dt.strftime('%H:%M:%S')) # Extrai apenas o tempo da coluna de data e hora
    
    print('Coluna de horário de chegada programado corrigida.')
    return df

df = fix_ScheduledArrivalTime_column(df)
display(df.head(15))
display(df.info())

In [ ]:
def fix_RecordedAtTime(each_row):
    real = each_row['RecordedAtTime']
    prog = each_row['ScheduledArrivalTime']
    
    # Se real for menor que programado e a diferença for maior que 12h, então virou o dia
    if real < prog and (prog - real) > pd.Timedelta(hours=12):
        return real + pd.Timedelta(days=1)
    else:
        return real

def fix_RecordedAtTime_column(df):
    df['RecordedAtTime'] = df.apply(fix_RecordedAtTime, axis=1)
    
    print('Intervalo de dias em horários realizados após a meia-noite ajustado para o dia seguinte com base no horário programado.')
    return df

df = fix_RecordedAtTime_column(df)
display(df.head(15))

In [ ]:
def add_DiffArrivalMins_column(df): # Função para adicionar a coluna de diferença entre a chegada programada e a chegada real
    df['DiffArrivalMins'] = (df['RecordedAtTime'] - df['ScheduledArrivalTime']).dt.total_seconds()/60 # Converte a diferença de tempo para minutos
    df['DiffArrivalMins'] = df['DiffArrivalMins'].astype('int64')  # Converte a diferença de tempo para minutos
    
    print('Coluna de entre a chegada programada e a chegada real adicionada.')
    return df

df = add_DiffArrivalMins_column(df)
display(df.sample(15))
display(df.info())


In [ ]:
def add_Recorded_and_ScheduledTimeRange_columns(df): # Função para adicionar colunas de faixa horária do horário registrado e programado
    df['RecordedTimeRange'] = df['RecordedAtTime'].dt.components['hours'].astype('int64') # Extrai apenas a hora da coluna de horário registrado
    df['ScheduledTimeRange'] = df['ScheduledArrivalTime'].dt.components['hours'].astype('int64') # Extrai apenas a hora da coluna de horário programado

    print("Coluna de faixa horária do registrado e programado adicionadas.")
    return df

df = add_Recorded_and_ScheduledTimeRange_columns(df)
display(df[df['RecordedTimeRange'] != df['ScheduledTimeRange']].sample(15)) # Exibe amostra de registros onde a faixa horária do registrado é diferente do programado
display(df.info())

## Correção de Tipos das demais colunas

In [ ]:
def change_data_types(df): # Função para alterar os tipos de dados das colunas do DataFrame
    df = df.astype({'DirectionRef': 'int64',  # Converte algumas colunas para o tipo certo
                    'PublishedLineName': 'string',
                    'VehicleRef': 'string'})

    print("Tipos de dados das colunas restantes alterados.")
    return df

df = change_data_types(df)
display(df.info())

# Salvamento dos dados transformados e upload pra Cloud

In [ ]:
file_name_silver = f"mta_{year_month}_cleaned.csv"

In [ ]:
def save_to_csv_silver(df, path_folder_silver, file_name_silver): # Função para salvar o DataFrame limpo no diretório silver
    df.to_csv(os.path.join(path_folder_silver, file_name_silver), index=False) # Salva o DataFrame limpo no silver

    print(f'Arquivo "{file_name_silver}" salvo no diretório "silver/".')

save_to_csv_silver(df, path_folder_silver, file_name_silver)

In [ ]:
def upload_to_bucket_silver(path_folder_silver, file_name_silver): # Função para enviar o arquivo limpo para o bucket do GCP
    client = storage.Client() # Cria o cliente para acessar o bucket
    bucket = client.bucket('etl_bus_gps') # Referência pro bucket do GCP
    
    path_folder_file_silver = os.path.join(path_folder_silver, file_name_silver) # Caminho completo do arquivo a ser enviado
    
    blob = bucket.blob(f'temp/silver/{file_name_silver}') # Cria o blob no bucket
    blob.upload_from_filename(path_folder_file_silver) # Faz o upload do arquivo para o bucket

    print(f'Arquivo "{file_name_silver}" enviado para a pasta "silver/" no bucket "{bucket.name}"')

upload_to_bucket_silver(path_folder_silver, file_name_silver) # Envia o arquivo limpo para o bucket do GCP

# Pipeline de Execução

In [ ]:
def pipeline_silver(path_folder_bronze, file_name_bronze, path_folder_silver, file_name_silver): # Função que executa todo o pipeline de limpeza, transformação, salvamento e upload
    print("Iniciando o pipeline Silver...")
    df = clean_and_filter_data(path_folder_bronze, file_name_bronze)  # Limpa e filtra os dados
    df = add_RecordedAtDate_and_Time_columns(df)  # Adiciona colunas de data e de hora
    df = fix_ScheduledArrivalTime_column(df)  # Corrige a coluna de horário programado
    df = fix_RecordedAtTime_column(df)  # Corrige a parte de dias do intervalo de horário registrado
    df = add_DiffArrivalMins_column(df)  # Adiciona coluna de diferença de tempo entre horário registrado e programado
    df = add_Recorded_and_ScheduledTimeRange_columns(df)  # Adiciona colunas de faixa horária para horário registrado e programado
    df = change_data_types(df)  # Altera os tipos de dados das colunas restantes
    save_to_csv_silver(df, path_folder_silver, file_name_silver)  # Salva o DataFrame limpo no silver
    upload_to_bucket_silver(path_folder_silver, file_name_silver)  # Envia o arquivo limpo para o bucket do GCP

    print(f'Pipeline Silver executado com sucesso para o arquivo "{file_name_bronze}"!!\n')
    
pipeline_silver(path_folder_bronze, file_name_bronze, path_folder_silver, file_name_silver)  # Executa o pipeline completo

In [ ]:
year_month_list = [] # Lista para armazenar os anos e meses

year_month_list.append(extract_year_month(pd.to_datetime('2017-06-01'))) # Adiciona o ano e mês de junho de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-08-01'))) # Adiciona o ano e mês de agosto de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-10-01'))) # Adiciona o ano e mês de outubro de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-12-01'))) # Adiciona o ano e mês de dezembro de 2017

for year_month in year_month_list: # Itera sobre os anos e meses na lista, simulando o Airflow
    file_name_bronze = f'mta_{year_month}.csv'  # Define o nome do arquivo bronze com base no ano e mês
    file_name_silver = f"mta_{year_month}_cleaned.csv"  # Define o nome do arquivo silver com base no ano e mês
    pipeline_silver(path_folder_bronze, file_name_bronze, path_folder_silver, file_name_silver)  # Executa o pipeline para o ano e mês especificados